In [5]:
import scipy
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [13]:
L = 5 #system size
M = 0.1 #fermion mass
N=2#number of layers
l=1 #spin length
m_epsilon = 0
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [14]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
       0.         |2 2 2 2 2>           242  
       1.         |2 2 2 2 1>           241  
       2.         |2 2 2 2 0>           240  
       3.         |2 2 2 1 2>           239  
       4.         |2 2 2 1 1>           238  
       5.         |2 2 2 1 0>           237  
       6.         |2 2 2 0 2>           236  
       7.         |2 2 2 0 1>           235  
       8.         |2 2 2 0 0>           234  
       9.         |2 2 1 2 2>           233  
      10.         |2 2 1 2 1>           232  
      11.         |2 2 1 2 0>           231  
      12.         |2 2 1 1 2>           230  
      13.         |2 2 1 1 1>           229  
      14.         |2 2 1 1 0>           228  
      15.         |2 2 1 0 2>           227  
      16.         |2 2 1 0 1>           226  
      17.         |2 2 1 0 0>           225  
      18.         |2 2 0 2 2>           224  
      19.         |2 2 0 2 1>           223  
      20.  

In [15]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []
gauss_law_matrix = []
z_comp_minf = [[-m_epsilon,0]]
z_comp_pinf = [[m_epsilon,L-1]]

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]],
            ["I",z_comp_minf],

        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",z_comp_pinf],
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]],
            ["I",z_comp_pinf],

        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
for i in range(L+1):
    gauss_law_matrix.append(sparse.dok_matrix(Gauss_law[i].toarray()))

In [16]:
#projectors on g=1 for every site
Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for j in range(basis.Ns):
        if (np.real(gauss_law_matrix[i][j,j])>=1.-1e-8) & (np.real(gauss_law_matrix[i][j,j])<=1.+1e-8):
            Proj[i][j][j] = 1.
for i in range(L+1):
    Proj[i] = sparse.csc_matrix(Proj[i])

In [17]:
A = np.zeros((2*l+1,2*l+1))
for i in range(2*l+1):
    for j in range(2*l+1):
        A[i][j] = (-l+i)**j
b = np.zeros(2*l+1)
for i in range(2*l+1):
    b[i] = (-1)**(-l+i)
x = scipy.linalg.solve(A,b)
print(x)

[ 1.  0. -2.]


In [25]:
const_term = []
linear_term = []
quadratic_term = []
#cubic_term = []
#quartic_term = []
#quintic_term = []
#sextic_term = []
interaction_p = []
interaction_m = []

for i in range(L-1):
    const_term.append([[x[0]/(2*np.sqrt(l*(l+1))),i]])
    linear_term.append([[x[1]/(2*np.sqrt(l*(l+1))),i,i+1]])
    quadratic_term.append([[x[2]/(2*np.sqrt(l*(l+1))),i,i+1,i+1]])
    #cubic_term.append([[((-1)**i*x[3])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1]])
    #quartic_term.append([[((-1)**i*x[4])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1]])
    #quintic_term.append([[((-1)**i*x[5])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1]])
    #sextic_term.append([[((-1)**i*x[6])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1,i+1]])


    interaction_p.append([
        ["+", const_term[i]],
        ["+z", linear_term[i]],
        ["+zz", quadratic_term[i]],
        #["+zzz", cubic_term[i]],
        #["+zzzz", quartic_term[i]],
        #["+zzzzz", quintic_term[i]],
        #["+zzzzzz", sextic_term[i]],

    ])

    interaction_m.append([
        ["-", const_term[i]],
        ["-z", linear_term[i]],
        ["-zz", quadratic_term[i]],
        #["-zzz", cubic_term[i]],
        #["-zzzz", quartic_term[i]],
        #["-zzzzz", quintic_term[i]],
        #["-zzzzzz", sextic_term[i]],

    ])

const_term.append([[(0.5*(-1)**m_epsilon)/np.sqrt(l*(l+1)),L-1]])

interaction_p.append([
    ["+", const_term[L-1]]
])
interaction_m.append([
    ["-", const_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [np.zeros((basis.Ns,basis.Ns)) for i in range(L)]
h_int = np.zeros((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].toarray()@Proj[i+1]+Proj[i+1]@H_int_m[i].toarray()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.toarray()

h_int = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints

G=0
for i in range(L+1):
    G += 10.*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.toarray()
constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
print(full_ham_matrix)

[[2.7+0.j 0. +0.j 0. +0.j ... 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 2. +0.j 0. +0.j ... 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 2.3+0.j ... 0. +0.j 0. +0.j 0. +0.j]
 ...
 [0. +0.j 0. +0.j 0. +0.j ... 2.7+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j ... 0. +0.j 2. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j ... 0. +0.j 0. +0.j 2.3+0.j]]


In [26]:
#basis vectors as arrays
#basis_vectors = []
#for index in range(basis.Ns):
#    state = np.zeros(basis.Ns)
#    state[basis.Ns-index-1] = 1.
#    state = state.tolist()
#    basis_vectors.append(state)

In [27]:
#projectors on g=1 for every site
#Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
#for i in range(L+1):
#    for state in basis_vectors:
#        state = np.array(state)
#        if (Gauss_law[i].expt_value(state)>=1.-1e-8) & (Gauss_law[i].expt_value(state)<=1.+1e-8):
#            Proj[i] += np.outer(state,state)

In [28]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = np.linalg.eigh(full_ham_matrix)
print(eigenval[eigenval<=1e1])
print(eigenvec[:,0])

[-1.0416023  -0.34551385 -0.34551385 -0.22065556 -0.13898669 -0.13898669
 -0.0142565   0.15174042  0.16101331  0.16101331  0.33967222  0.33967222
  0.33967222  0.33967222  0.33967222  0.35848584  0.53430948  0.53430948
  0.53479822  0.55681029  0.56101331  0.56101331  0.6         0.6
  0.6         0.63967222  0.63967222  0.69336559  0.69336559  0.72748942
  0.72748942  0.73967222  0.73967222  0.9         0.9         0.9
  0.9         0.9         0.9         0.9         0.9         0.93430948
  0.93430948  0.99336559  0.99336559  1.          1.          1.
  1.          1.          1.          1.03967222  1.03967222  1.03967222
  1.03967222  1.09336559  1.09336559  1.10404012  1.11592807  1.2
  1.2         1.2         1.2         1.2         1.2284373   1.276925
  1.276925    1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.3         1.3         1.3         1.3         1.3         1.3
  1.39336559  1.39336559  1.43

In [29]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = np.linalg.eigh(constrained_full_ham_matrix)

In [30]:
#Gauge invariant part of the spectrum
print(eigenvalues[eigenvalues<=1e1])
print(eigenvectors[:,0])

[-1.0416023  -0.0142565   0.15174042  0.35848584  0.53479822  0.55681029
  1.10404012  1.11592807  1.2284373   1.48612285  1.75182104  1.76913858
  1.95519133  2.01107024  2.09579725  2.34850063  2.71262995  2.85799378
  3.6173529 ]
[-4.11576340e-03+0.j  4.85722573e-17+0.j  8.32667268e-17+0.j
 -1.53995498e-02+0.j  2.87872606e-02+0.j -1.11022302e-16+0.j
  0.00000000e+00+0.j -2.77555756e-17+0.j  1.38777878e-17+0.j
  0.00000000e+00+0.j -4.33680869e-19+0.j  0.00000000e+00+0.j
  1.69406589e-20+0.j  2.03287907e-20+0.j -2.71050543e-20+0.j
 -3.38813179e-21+0.j  0.00000000e+00+0.j  3.17637355e-22+0.j
  0.00000000e+00+0.j -2.11758237e-22+0.j  0.00000000e+00+0.j
 -7.44462551e-24+0.j -9.82276977e-25+0.j -8.48789090e-25+0.j
  1.22967638e-26+0.j -3.73180906e-25+0.j -3.46359241e-30+0.j
 -1.53995498e-02+0.j  2.27203315e-18+0.j  3.49118661e-18+0.j
 -7.30952282e-02+0.j  1.42446890e-01+0.j -2.79968679e-18+0.j
  3.45249060e-19+0.j  8.52945057e-20+0.j  4.83832563e-20+0.j
  0.00000000e+00+0.j  3.00926554e-3

In [31]:
#initial state
string = ""
for i in range(L):
    string +="1"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.

In [32]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

0.0


In [42]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

(2.3+0j)


In [43]:
def rot_sigma(k,i,j,phi,theta):
    if k>=L:
        print("Warning! Rotations apply to k<=L-1!")
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,sparse.identity(2*l+1))+scipy.sparse.kron(sparse.identity(2*l+1),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k,i,j,phi,theta)@sigma_o
    return(sigma_o)

def rot_sigma_all(i,j,phi,theta):
    sigma_all = sparse.identity(basis.Ns,dtype = complex)
    for k in range(L):
        sigma_all = rot_sigma(k,i,j,phi,theta)@sigma_all
    return(sigma_all)

In [44]:
def ms_hamiltonian(i,j,phi):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma[2*l-i][2*l-i] = -0.2

    sigma = sparse.csc_matrix(sigma)
    summe = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for k in range(L):
        tensprod = 0
        if k == 0:
            tensprod = sigma
            for m in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        summe +=tensprod.toarray()
    return summe@summe
                
def sigma_z_part(i,j):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-i] = 1.
    sigma[2*l-j][2*l-j] = -1.
    sigma = sparse.csc_matrix(sigma)
    summe = 0
    for k in range(L):
        tensprod = 0
        if k==0:
            tensprod = sigma
            for i in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        summe +=tensprod
    return summe
    
matrix_part = ms_hamiltonian(0,2,0)
ms_ham_eigenval, ms_ham_eigenvec = scipy.linalg.eigh(matrix_part)




def ms_gate_global(i,j,theta):
    exponential = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        exponential[i][i] = np.exp(1j*ms_ham_eigenval[i]*theta/4)
    exponential = scipy.sparse.csc_matrix(exponential)
    return ms_ham_eigenvec@exponential@ms_ham_eigenvec.T.conj()

    

In [45]:
#import timeit
#print(timeit.timeit('[ms_gate_global(0,2,np.pi)]', globals=globals()))
import datetime
print(datetime.datetime.now())
a = ms_gate_global(0,2,np.pi)
print(datetime.datetime.now())


2022-03-22 09:47:59.844964
2022-03-22 09:48:01.546156


In [355]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [356]:
full_ham_matrix = sparse.csc_matrix(full_ham_matrix)

In [ ]:
N=1
import optuna
def cost_function_sigma(trial):
    psi_var = psi_0
    for i in range(N):
        for k in range(18):
            theta[18*i+k] = trial.suggest_float("18*"+str(i)+"+"+str(k), -5*np.pi, 5*np.pi)
        psi_var = rot_ms(0,0,2,0,theta[18*i+5])@rot_ms(0,1,2,0,theta[18*i+4])@rot_ms(0,0,1,0,theta[18*i+3])@rot_sigma(0,0,2,0,theta[18*i+2])@rot_sigma(0,1,2,0,theta[18*i+1])@rot_sigma(0,0,1,0,theta[18*i+0])@psi_var
        psi_var = rot_ms_o(0,2,0,theta[18*i+17])@rot_ms_o(1,2,0,theta[18*i+16])@rot_ms_o(0,1,0,theta[18*i+15])@rot_ms_e(0,2,0,theta[18*i+14])@rot_ms_e(1,2,0,theta[18*i+13])@rot_ms_e(0,1,0,theta[18*i+12])@rot_sigma_all(0,2,0,theta[18*i+11])@rot_sigma_o(1,2,0,theta[18*i+10])@rot_sigma_e(0,1,0,theta[18*i+9])@rot_sigma_all(0,2,0,theta[18*i+8])@rot_sigma_e(1,2,0,theta[18*i+7])@rot_sigma_e(0,1,0,theta[18*i+6])@psi_var
        psi_var = rot_ms(L-2,0,2,0,theta[18*i+5])@rot_ms(L-2,1,2,0,theta[18*i+4])@rot_ms(L-2,0,1,0,theta[18*i+3])@rot_sigma(L-1,0,2,0,theta[18*i+2])@rot_sigma(L-1,1,2,0,theta[18*i+1])@rot_sigma(L-1,0,1,0,theta[18*i+0])@psi_var
    cost = np.conj(psi_var)@constrained_full_ham_matrix@psi_var
    return np.real(cost)

if __name__ == "__main__":
    sampler = optuna.samplers.CmaEsSampler(restart_strategy="ipop",
        inc_popsize=2)
    study = optuna.create_study(sampler=sampler)
    study.optimize(cost_function_sigma, n_trials=3000)

/home/ppopov/anaconda3/lib/python3.8/site-packages/optuna/samplers/_cmaes.py:221: ExperimentalWarning: `restart_strategy` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2022-03-21 17:53:31,327] A new study created in memory with name: no-name-8180cf7b-0ef0-49eb-9421-8d98bbe9a75d
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 17:53:31,482] Trial 0 finished with value: 1.3775825057711932 and parameters: {'18*0+0': -12.858186181030629, '18*0+1': -3.271300061491564, '18*0+2': 1.4420801101269163, '18*0+3': 5.113803590628454, '1

/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 17:53:32,549] Trial 7 finished with value: 2.5484091981973327 and parameters: {'18*0+0': 3.940876455719012, '18*0+1': -0.7652245634320306, '18*0+2': -7.555716291322449, '18*0+3': 2.9067038523467312, '18*0+4': -5.618939979975335, '18*0+5': 3.029952093358017, '18*0+6': -6.567201225252094, '18*0+7': -1.1335802562436046, '18*0+8': -5.573129702939739, '18*0+9': 0.4377077799659985, '18*0+10': 7.49388933797888, '18*0+11': 0.35099278359998254, '18*0+12': 3.7663585174343175, '18*0+13': -0.832202404852457, '18*0+14': -4.8860678165597

[I 2022-03-21 17:53:33,620] Trial 14 finished with value: 4.1021498524823645 and parameters: {'18*0+0': -5.130958395626683, '18*0+1': 10.818060640231623, '18*0+2': 1.743224729771386, '18*0+3': -10.942138589724404, '18*0+4': 2.6771144730500205, '18*0+5': 10.066246115778913, '18*0+6': -2.4489222965213813, '18*0+7': 5.049283072083991, '18*0+8': 3.3025924703777063, '18*0+9': -5.286284064671455, '18*0+10': -0.1856067364934355, '18*0+11': -0.0837202157204624, '18*0+12': -7.507285229313769, '18*0+13': 2.8019052630125647, '18*0+14': 6.572594082849312, '18*0+15': 4.819548976638579, '18*0+16': -5.045451972467867, '18*0+17': 4.559158983650498}. Best is trial 0 with value: 1.3775825057711932.
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:2

/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 17:53:34,903] Trial 22 finished with value: 9.083426777643158 and parameters: {'18*0+0': 5.104750172137316, '18*0+1': -4.933006900400043, '18*0+2': 13.537373357902782, '18*0+3': -2.6663533546923825, '18*0+4': 4.7768382168968655, '18*0+5': -4.449335368172893, '18*0+6': -9.666734914045069, '18*0+7': 5.525160495050138, '18*0+8': 2.1377767670502763, '18*0+9': 8.447992985405936, '18*0+10': -1.535917971760462, '18*0+11': -2.1271562644950617, '18*0+12': -7.939055244741401, '18*0+13': -5.919028890742869, '18*0+14': -8.0438130660255

[I 2022-03-21 17:53:36,009] Trial 29 finished with value: 4.907588740207563 and parameters: {'18*0+0': -8.27393565940827, '18*0+1': 1.943499466803079, '18*0+2': 4.25755152638078, '18*0+3': -0.23971089374121357, '18*0+4': -0.07919092225133229, '18*0+5': -10.580096631569546, '18*0+6': 6.544092908298652, '18*0+7': 1.7394363537886959, '18*0+8': -1.3428764826193071, '18*0+9': 1.888313320320668, '18*0+10': 0.9648839183658706, '18*0+11': 9.100173243288697, '18*0+12': -6.434231217354041, '18*0+13': 0.8466826881411196, '18*0+14': -3.780648046350074, '18*0+15': 11.000079140004438, '18*0+16': 0.2794827575737112, '18*0+17': -5.2540434510259075}. Best is trial 0 with value: 1.3775825057711932.
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:2

...

/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 18:01:29,030] Trial 2858 finished with value: -0.7020411809330531 and parameters: {'18*0+0': -7.732116373925594, '18*0+1': 5.943144179246014, '18*0+2': -12.111791281782626, '18*0+3': 11.168459395835088, '18*0+4': 3.0006672628651536, '18*0+5': -7.878832685049768, '18*0+6': -4.776141000616681, '18*0+7': -10.889251923983105, '18*0+8': 0.9164811813179384, '18*0+9': 6.247235635053307, '18*0+10': 0.02516562706637493, '18*0+11': 15.669700271468674, '18*0+12': -5.774017175570269, '18*0+13': -15.41521107093423, '18*0+14': 15.3065263

[I 2022-03-21 18:01:30,223] Trial 2865 finished with value: -0.703928122841033 and parameters: {'18*0+0': -7.763469349037873, '18*0+1': 6.019781808679186, '18*0+2': -12.155856873218914, '18*0+3': 11.28209812009168, '18*0+4': 3.024854551215688, '18*0+5': -7.7673024992137405, '18*0+6': -4.653407698512194, '18*0+7': -10.829981916730837, '18*0+8': 0.9100412474341975, '18*0+9': 6.4091878821329065, '18*0+10': 0.056567187326894905, '18*0+11': 15.65278076077965, '18*0+12': -5.7449857855999005, '18*0+13': -15.67389723126323, '18*0+14': 15.252223589405924, '18*0+15': 11.294296594648577, '18*0+16': 3.9978940084911487, '18*0+17': 12.41367835576487}. Best is trial 2850 with value: -0.7089896117517308.
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linso

/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 18:01:31,581] Trial 2873 finished with value: -0.7044093449852302 and parameters: {'18*0+0': -7.7313346742158755, '18*0+1': 5.9746243939958195, '18*0+2': -12.136152434872566, '18*0+3': 11.172694378167188, '18*0+4': 3.0385560484414205, '18*0+5': -7.819705418946425, '18*0+6': -4.7144997512978986, '18*0+7': -10.94329012332875, '18*0+8': 0.9074336421767507, '18*0+9': 6.4923411265104916, '18*0+10': 0.009298831675929896, '18*0+11': 15.6782530905177, '18*0+12': -5.682657535935577, '18*0+13': -15.462949951285347, '18*0+14': 15.5051

/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
[I 2022-03-21 18:01:32,967] Trial 2881 finished with value: -0.7079571748008846 and parameters: {'18*0+0': -7.768792605867489, '18*0+1': 6.0905702010448755, '18*0+2': -12.183989512958023, '18*0+3': 11.468043492702938, '18*0+4': 2.9237854536857637, '18*0+5': -7.826631156749801, '18*0+6': -4.683213725676877, '18*0+7': -10.775457448649515, '18*0+8': 0.8915089500845865, '18*0+9': 6.475427781851855, '18*0+10': 0.00804101292082423, '18*0+11': 15.65399929343579, '18*0+12': -5.586967824826682, '18*0+13': -15.464563989326464, '18*0+14': 15.443652

[I 2022-03-21 18:01:34,159] Trial 2888 finished with value: -0.701556739302591 and parameters: {'18*0+0': -7.819549552033909, '18*0+1': 6.154028404213167, '18*0+2': -12.295423010355774, '18*0+3': 11.46735392635039, '18*0+4': 3.0155222026208213, '18*0+5': -7.574648077664391, '18*0+6': -4.66530963064218, '18*0+7': -10.709576174707063, '18*0+8': 1.017122525143199, '18*0+9': 6.564583627253685, '18*0+10': -0.01876395744398493, '18*0+11': 15.362202373629055, '18*0+12': -5.641224348000463, '18*0+13': -15.266631301059027, '18*0+14': 15.370225649368058, '18*0+15': 11.377609516556483, '18*0+16': 3.8974910140920094, '18*0+17': 12.36459040837668}. Best is trial 2850 with value: -0.7089896117517308.
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/ppopov/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolv

In [336]:
psi_var = psi_0
theta = np.zeros(18*N)
i=0
for value in study.best_trial.params.items():
    theta[i] = value[1]
    i+=1
for i in range(N):
    psi_var = rot_ms(0,0,2,0,theta[18*i+5])@rot_ms(0,1,2,0,theta[18*i+4])@rot_ms(0,0,1,0,theta[18*i+3])@rot_sigma(0,0,2,0,theta[18*i+2])@rot_sigma(0,1,2,0,theta[18*i+1])@rot_sigma(0,0,1,0,theta[18*i+0])@psi_var
    psi_var = rot_ms_o(0,2,0,theta[18*i+17])@rot_ms_o(1,2,0,theta[18*i+16])@rot_ms_o(0,1,0,theta[18*i+15])@rot_ms_e(0,2,0,theta[18*i+14])@rot_ms_e(1,2,0,theta[18*i+13])@rot_ms_e(0,1,0,theta[18*i+12])@rot_sigma_all(0,2,0,theta[18*i+11])@rot_sigma_o(1,2,0,theta[18*i+10])@rot_sigma_e(0,1,0,theta[18*i+9])@rot_sigma_all(0,2,0,theta[18*i+8])@rot_sigma_e(1,2,0,theta[18*i+7])@rot_sigma_e(0,1,0,theta[18*i+6])@psi_var
    psi_var = rot_ms(L-2,0,2,0,theta[18*i+5])@rot_ms(L-2,1,2,0,theta[18*i+4])@rot_ms(L-2,0,1,0,theta[18*i+3])@rot_sigma(L-1,0,2,0,theta[18*i+2])@rot_sigma(L-1,1,2,0,theta[18*i+1])@rot_sigma(L-1,0,1,0,theta[18*i+0])@psi_var
print(np.abs(np.dot(np.conj(psi_var),eigenvectors[:,0]))**2)

0.8848333247191719


In [ ]:
theta = np.zeros(5*N)

import optuna
#cost function definition
def cost_function_sigma(trial):
    psi_var = psi_0
    for i in range(N):
        theta[5*i+0] = trial.suggest_float("5*"+str(i)+"+0", -3*np.pi, 3*np.pi)
        theta[5*i+1] = trial.suggest_float("5*"+str(i)+"+1", -3*np.pi, 3*np.pi)
        theta[5*i+2] = trial.suggest_float("5*"+str(i)+"+2", -3*np.pi, 3*np.pi)
        theta[5*i+3] = trial.suggest_float("5*"+str(i)+"+3", -3*np.pi, 3*np.pi)
        theta[5*i+4] = trial.suggest_float("5*"+str(i)+"+4", -3*np.pi, 3*np.pi)


        psi_var = ms_gate_global(0,2,theta[5*i+0])@psi_var
        psi_var = rot_sigma_e(0,2,0,theta[5*i+2])@rot_sigma_e(0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma_o(0,2,0,theta[5*i+4])@rot_sigma_o(0,1,0,theta[5*i+3])@psi_var
        psi_var = rot_sigma(0,0,2,0,theta[5*i+2])@rot_sigma(0,0,1,0,theta[5*i+1])@psi_var
        psi_var = rot_sigma(L-1,0,2,0,theta[5*i+2])@rot_sigma(L-1,0,1,0,theta[5*i+1])@psi_var
        
    cost = np.conj(psi_var)@full_ham_matrix@psi_var
    return np.real(cost)

if __name__ == "__main__":
    sampler = optuna.samplers.CmaEsSampler()
    study = optuna.create_study(sampler=sampler)
    study.optimize(cost_function_sigma, n_trials=3000)